In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime as dt
import io, os, json, pickle, gzip
import re, pyarrow, boto3

In [7]:
from sklearn.metrics import mean_absolute_percentage_error as mape, mean_squared_error as mse
#from sklearn.model_selection import cross_val_score as cv
def wmape(actual, pred):
    wmape=sum(abs(actual-pred))/sum(actual)
    return wmape

# from here if model not exist

In [9]:
# load training data locally
def load_eac_table(data_file):  
    s3 = boto3.resource('s3')  
    bucket = s3.Bucket('innovation-dev-athena-results')
    prefix_objs = bucket.objects.filter(Prefix='tables/'+data_file)
    df = []
    for obj in prefix_objs:
        body = obj.get()['Body'].read()
        temp = pd.read_parquet(io.BytesIO(body))
        df.append(temp)
    df_total = pd.concat(df)
    df_total.reset_index(drop=True, inplace=True)
    return df_total

In [12]:
df_sample_train = load_eac_table('sample training data/d88cc9bd-97a6-44f8-a36b-4e2b0ae48f17/')
df_sample_test = load_eac_table('sample training data/917e6ae0-d77e-4edf-816c-b651c26a6783/')

In [14]:
df_train_x, df_train_y = df_sample_train.loc[:, ['sum_dpc', 'cal_aa', 'previous_eac']].astype('float'), df_sample_train.loc[:,'current_eac'].astype('float')
df_test_x, df_test_y = df_sample_test.loc[:, ['sum_dpc', 'cal_aa', 'previous_eac']].astype('float'), df_sample_test.loc[:,'current_eac'].astype('float')

In [ ]:
del df_sample_train, df_sample_test

In [17]:
from xgboost import XGBRegressor

def xgb(train_x, train_y):
    model = XGBRegressor(n_estimators=200, max_depth=5, eval_metric=mape)
    model.fit(train_x, train_y)
    return model  


In [8]:
xgb_model = xgb(df_train_x, df_train_y)

In [32]:
del df_train_x, df_train_y, df_test_x, df_test_y

# from here if model exists

In [3]:
#pickle.dump(xgb_model, open('model.pkl', 'wb'))
xgb_model = pickle.load(open('model.pkl', 'rb'))

In [ ]:
"""
xgb_pred = xgb_model.predict(df_train_x)
score_xgb = cv(xgb_model, df_train_x, df_train_y, cv=5)  
mse_xgb =  mse(xgb_pred, df_train_y)
mape_xgb =  mape(xgb_pred, df_train_y)
wmape_xgb = wmape(xgb_pred, df_train_y)
print("XGB training MSE: {}, training MAPE: {}, training WMAPE: {} \n".format(mse_xgb,mape_xgb,wmape_xgb))
print("Validation scores for XGB: {} \n".format(score_xgb))
"""

In [10]:
"""
xgb_pred_test = xgb_model.predict(df_test_x)
mse_xgb_test = mse(xgb_pred_test, df_test_y)
mape_xgb_test = mape(xgb_pred_test, df_test_y)
wmape_xgb_test = wmape(xgb_pred_test, df_test_y)
print("XGB testing MSE: {}, testing1 MAPE: {}, testing1 WMAPE: {} \n".format(mse_xgb_test,mape_xgb_test,wmape_xgb_test))
"""

XGB testing MSE: 7488078.878932534, testing1 MAPE: 0.052439136250737914, testing1 WMAPE: 0.0595960765010869 



In [14]:
"""
model_file_name = "XGBoost-regression-model"
xgb_model.save_model(model_file_name)
!tar czvf model.tar.gz $model_file_name
"""

/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/xgboost/sklearn.py:761: UserWarning: eval_metric is not saved in Scikit-Learn meta.
  warnings.warn(


XGBoost-regression-model


In [4]:
#batch = pd.read_parquet('s3://innovation-dev-athena-results/tables/all-batches/27e0f1ea-03b6-44ae-93d7-2a81d4522cab/')
#batch = pd.read_parquet('s3://innovation-dev-athena-results/tables/all-batches/2d86dd65-bbbf-4bdb-a530-665e5e678f1c/')
#batch = pd.read_parquet('s3://innovation-dev-athena-results/tables/all-batches/352a44b6-d94e-4a38-8fcf-d78d9921acb7/')
#batch = pd.read_parquet('s3://innovation-dev-athena-results/tables/all-batches/3ca8402d-8e96-4c86-aeed-6a9be1aa9982/')
#batch = pd.read_parquet('s3://innovation-dev-athena-results/tables/all-batches/4b41085a-70df-43d5-9514-2cb973c501b7/')
#batch = pd.read_parquet('s3://innovation-dev-athena-results/tables/all-batches/4ccf9688-a60d-446a-a9d1-40c634263325/')
#batch = pd.read_parquet('s3://innovation-dev-athena-results/tables/all-batches/5e2f80fc-14e0-40e2-bc03-567dcfa18402/')
#batch = pd.read_parquet('s3://innovation-dev-athena-results/tables/all-batches/d9d1ba4b-7057-4e9e-84f1-26b39d283297/')
#batch = pd.read_parquet('s3://innovation-dev-athena-results/tables/all-batches/dde75b82-2049-4e60-8700-23f3a248d164/')
batch = pd.read_parquet('s3://innovation-dev-athena-results/tables/all-batches/e05f9833-c0f2-4c29-8783-a81bff20a45f/')

In [5]:
batch_x, batch_y = batch.loc[:, ['sum_dpc', 'cal_aa', 'previous_eac']].astype('float'), batch.loc[:,'current_eac'].astype('float')

In [8]:
xgb_batch_pred = xgb_model.predict(batch_x)
mse_xgb_batch = mse(xgb_batch_pred, batch_y)
mape_xgb_batch = mape(xgb_batch_pred, batch_y)
wmape_xgb_batch = wmape(xgb_batch_pred, batch_y)
print("XGB testing MSE: {}, testing MAPE: {}, testing WMAPE: {} \n".format(mse_xgb_batch,mape_xgb_batch,wmape_xgb_batch))

XGB testing MSE: 15380665.2128787, testing MAPE: 0.05202811409285556, testing WMAPE: 0.05911141826790941 



In [9]:
xgb_batch_pred[2300:2310]

array([3632.9138, 5010.4585, 4802.1343, 4760.0205, 3502.5322, 5508.182 ,
       5525.8696, 5532.6577, 5720.129 , 5669.4165], dtype=float32)

In [10]:
batch_y[2300:2310]

2300    4983.9
2301    4984.0
2302    4983.9
2303    3897.4
2304    3689.9
2305    5544.9
2306    5531.9
2307    5567.5
2308    5567.3
2309    5690.7
Name: current_eac, dtype: float64

# if need to batch transform and upload to s3

In [18]:
new_batch = pd.concat((batch, pd.DataFrame(xgb_batch_pred, columns = ['cal_eac'])), axis=1)

In [28]:
del batch, batch_x, batch_y, xgb_batch_pred

NameError: name 'batch_x' is not defined

In [20]:
new_batch.head()

,rn,mpan,read_date,next_read_date,effective_from_settlement_date,effective_to_settlement_date,sum_dpc,cal_aa,previous_eac,current_eac,cal_eac
0,217978641,4400007407699,2022-05-15,2022-08-15,2022-05-15,2022-08-14,0.205177,1471.902268,1487.3,1395.7,1489.330200
1,217978642,4400007407699,2022-08-15,2022-11-15,2022-08-15,2022-11-14,0.228462,1299.995972,1395.7,1427.0,1386.020996
2,217978643,4400007407699,2022-11-15,2023-02-12,2022-11-15,2023-02-11,0.291652,1200.058730,1427.0,1369.0,1397.220947
3,217978644,4400007407699,2023-02-12,2023-02-13,2023-02-12,2023-02-12,0.003143,318.174931,1369.0,1369.0,1320.855103
4,217978645,4400007407850,2021-05-28,2022-02-07,2021-05-28,2022-02-06,0.690026,1175.173824,1375.4,1375.4,1380.804932


In [21]:
new_batch.to_parquet('batch_updated.parquet')
boto3.resource('s3').meta.client.upload_file('batch_updated.parquet','innovation-dev-athena-results', 'tables/all-batches-updated/batch10-updated')

In [23]:
del new_batch

# compare to conventional EAC calculation method

In [11]:
# conventional EAC equation
def conventional_eac(sum_dpc, aa, previous_eac):
    sum_dpc[sum_dpc>0.5]=0.5
    conventional_eac = aa*sum_dpc*2+previous_eac*(1-sum_dpc*2)
    return conventional_eac


In [12]:
conv_eac = conventional_eac(batch['sum_dpc'].astype('float'), batch['cal_aa'].astype('float'), batch['previous_eac'].astype('float'))

In [13]:
mape0 = mape(conv_eac, batch['current_eac'].astype('float'))
wmape0 = wmape(conv_eac, batch['current_eac'].astype('float'))
print("conventional EAC calculation error: mape: {}, wmape: {}".format(mape0, wmape0))
print("conventional EAC calculation error: mape: {}%, wmape: {}%".format(mape0*100, wmape0*100))

conventional EAC calculation error: mape: 0.17738166102762934, wmape: 0.31076651951966244
conventional EAC calculation error: mape: 17.738166102762936%, wmape: 31.076651951966245%
